# IMDB 영화평 감성분석(이진분류)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#from google.colab import files
#up = files.uplaod() # 파일이 커서 이거 느려,,, 그냥 끌어서 올리기! 아니다,, 똑같나..?

In [ ]:
# DataFrame으로 데이터 불러오기
df =pd.read_csv('labeledTrainData.tsv', sep='\t')
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [ ]:
# 따옴표 옵션주어 다시 불러오기
df =pd.read_csv('labeledTrainData.tsv', sep='\t', quoting=3) # quoting=3 따옴표 붗여서 
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [ ]:
df.review[0][:1000] # review 첫번째 데이터 1000 글자 

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

- 텍스트 전처리

In [ ]:
# <br /> : HTML 코드에서 줄바꿈 코드
# <br /> 태그 공백으로 변환하기

df.review = df.review.str.replace('<br />', '')
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [ ]:
# 구둣점, 숫자 제거 -> 영문자가 아닌 글자는 공백으로 변환 
# 영문자가 아닌 글자 = [^A-Za-z]
# 데이터프레임의 스트링 매서드는 정규표현식 사용 가능 

df.review = df.review.str.replace('[^A-Za-z]', ' ').str.strip()
df.review[0][:1000]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

1. Train/Test dataset 분리

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
    df.review, df.sentiment, stratify=df.sentiment, random_state=2022
)

y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

2. `CountVectorizer`로 변환 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words = 'english')
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_train_cv.shape

(18750, 65985)

In [ ]:
# test dataset은 train dataset을 변환한 Verctorizer로 변환해야 함! (순서 중요!!)
X_test_cv = cvect.transform(X_test)
X_test_cv.shape

(6250, 65985)

3. 분류
- 분류기: SVM

In [ ]:
# 이거 시간 너무 오래걸려서 넘어가자,,,,
from sklearn.svm import SVC
svc = SVC(random_state=2022)
%time svc.fit(X_train_cv, y_train) # %time 시간 측정 

분류기: `Naive Bayes`

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
%time nb.fit(X_train_cv, y_train)

CPU times: user 24.2 ms, sys: 994 µs, total: 25.2 ms
Wall time: 29.6 ms


MultinomialNB()

In [ ]:
nb.score(X_test_cv, y_test)

0.85488

In [ ]:
cvect2 = CountVectorizer(ngram_range=(1,2), stop_words='english')
X_train_cv2 = cvect2.fit_transform(X_train)
X_train_cv2.shape

(18750, 1387911)

In [ ]:
# 위와 동일한 방법?
cvect2 = CountVectorizer(ngram_range=(1,2), stop_words='english')
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_train_cv2.shape

(18750, 1387911)

In [ ]:
X_test_cv2 = cvect2.transform(X_test)
X_test_cv2.shape

(6250, 1387911)

In [ ]:
nb2 = MultinomialNB()
nb2.fit(X_train_cv2, y_train)

MultinomialNB()

In [ ]:
nb2.score(X_test_cv2, y_test)

0.86768

4. 모델 저장하고 불러오기


In [ ]:
import joblib
joblib.dump(cvect2, 'imdb_cvect12.pkl')
joblib.dump(nb2, 'imdb_nb2.pkl')
# 피클데이터는 다운로드 받아서 서버로 올리지 말기!!
# 이런 바이너리 파일은 깃헙 같은곳에 올리는거 아님~~

['imdb_nb2.pkl']

In [ ]:
# 저장됬는지 확인
!ls -l

total 114460
-rw-r--r-- 1 root root 39225378 Apr 28 02:51 imdb_cvect12.pkl
-rw-r--r-- 1 root root 44413869 Apr 28 02:51 imdb_nb2.pkl
-rw-r--r-- 1 root root 33556378 Apr 28 01:35 labeledTrainData.tsv
drwxr-xr-x 1 root root     4096 Apr 25 13:46 sample_data


In [ ]:
# 생성 후 저장한 모델 불러오기
new_cvect = joblib.load('imdb_cvect12.pkl')
new_nb = joblib.load('imdb_nb2.pkl')

## 실제 데이터로 검증

In [ ]:
review = """
Robert P is a terrible Batman. Absolutely no Bat-charisma at all. 
The Riddler character is a complete bore! I hated all of this film. 
I certainly do not understand the good reviews. It's not close to the Dark Knight caliber. 
Please do not make a sequel.
"""

In [ ]:
# 1) 텍스트 전처리
import re # string 데이터는 re.sub() 사용 
review = re.sub('[^A-Za-z]', ' ', review).strip()

In [ ]:
# 2) feature 변환 
review_cv = new_cvect.transform([review]) # review -> string 데이터라서 리스트로 만들어 줘야함 / new_cvect 위에서 만든 모델 
review_cv.shape

(1, 1387911)

In [ ]:
new_nb.predict(review_cv)

array([0])